In [ ]:
%matplotlib inline
import os
import time
import copy
import shutil
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models, datasets

import utils


plt.style.use("dark_background")

In [ ]:
training_timestamp = str(int(time.time()))
model_dir = f'trained_models/model_identification_resnext_{training_timestamp}/'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
shutil.copy2('./identification_resnext.ipynb', model_dir)

batch_size = 16

data_transforms = {'aligned_train': transforms.Compose([transforms.Resize(224),
                                                        transforms.RandomHorizontalFlip(),
                                                        transforms.ToTensor(),
                                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                   'aligned_val': transforms.Compose([transforms.Resize(224),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                  }

data_dir = 'data/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['aligned_train', 'aligned_val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, drop_last=True) for x in ['aligned_train', 'aligned_val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['aligned_train', 'aligned_val']}
class_names = image_datasets['aligned_train'].classes

device = torch.device("cuda:0")

real_batch, real_classes = next(iter(dataloaders['aligned_train']))
plt.figure(figsize=(8,8))
plt.imshow((np.transpose(real_batch[0].numpy(), (1, 2, 0))*np.array([0.229, 0.224, 0.225]))+np.array([0.485, 0.456, 0.406]))

In [ ]:
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
resnext50_32x4d.fc = nn.Linear(resnext50_32x4d.fc.in_features, 46)
resnext50_32x4d = resnext50_32x4d.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(resnext50_32x4d.parameters(), lr=0.001)

utils.print_and_log(model_dir, resnext50_32x4d)

In [ ]:
num_epochs = 3

unknown_id = 45

best_model = copy.deepcopy(resnext50_32x4d.state_dict())
best_acc = 0.0

train_losses = []
val_losses = []

utils.print_and_log(model_dir, f"{datetime.now()} Starting Training")

for epoch in range(num_epochs):
    for phase in ['aligned_train', 'aligned_val']:
        if phase == 'aligned_train':
            utils.print_and_log(model_dir, f"{datetime.now()} Epoch {epoch+1}/{num_epochs} Training")
            resnext50_32x4d.train()
        else:
            utils.print_and_log(model_dir, f"{datetime.now()} Epoch {epoch+1}/{num_epochs} Validation")
            resnext50_32x4d.eval()

        epoch_loss = 0.0
        epoch_acc = 0.0
        epoch_iter = 0
        
        epoch_unknown_correct = 0
        epoch_unknown_total = 0
        epoch_known_correct = 0
        epoch_known_total = 0
        
        for batch_index, (inputs, labels) in enumerate(dataloaders[phase]):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'aligned_train'):
                outputs = resnext50_32x4d(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                unknown_correct = torch.sum((preds == unknown_id) & (labels.data == unknown_id)).item()
                unknown_total = torch.sum(labels.data == unknown_id).item()
                known_correct = torch.sum(preds == labels.data).item() - unknown_correct
                known_total = inputs.size(0) - unknown_total

                if phase == 'aligned_train':
                    loss.backward()
                    optimizer.step()
                    
                    train_losses.append(loss.item())
                    if batch_index % 20 == 0:
                        utils.print_and_log(model_dir, f"{datetime.now()} [{epoch+1:02d}/{num_epochs}][{batch_index:04d}/{len(dataloaders[phase])}] "
                                            f"Loss:{loss.item():.4f} Acc:{torch.sum(preds == labels.data).item()/inputs.size(0):.4f} "
                                            f"Corrects(Known):{known_correct:02d}/{known_total:02d} "
                                            f"Corrects(Unknown):{unknown_correct:02d}/{unknown_total:02d}")
                
                epoch_loss += loss.item()
                epoch_acc += torch.sum(preds == labels.data).item()/inputs.size(0)
                epoch_iter += 1

                epoch_unknown_correct += unknown_correct
                epoch_unknown_total += unknown_total
                epoch_known_correct += known_correct
                epoch_known_total += known_total
        
        epoch_loss = epoch_loss / epoch_iter
        epoch_acc = epoch_acc / epoch_iter
        if phase == "aligned_train":
            utils.print_and_log(model_dir, f"{datetime.now()} [{epoch+1:02d}/{num_epochs}] Train Loss: {epoch_loss:.4f} Train Acc: {epoch_acc:.4f} "
                                f"Corrects(Known):{epoch_known_correct:04d}/{epoch_known_total:04d} Corrects(Unknown):{epoch_unknown_correct:04d}/{epoch_unknown_total:04d}")
        
        else:
            utils.print_and_log(model_dir, f"{datetime.now()} [{epoch+1:02d}/{num_epochs}] Val Loss: {epoch_loss:.4f} Val Acc: {epoch_acc:.4f} "
                                f"Corrects(Known):{epoch_known_correct:04d}/{epoch_known_total:04d} Corrects(Unknown):{epoch_unknown_correct:04d}/{epoch_unknown_total:04d}")
            val_losses.append(epoch_loss)
            
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(resnext50_32x4d.state_dict())

utils.print_and_log(model_dir, f"{datetime.now()} Best Val Acc:{best_acc:4f}")
resnext50_32x4d.load_state_dict(best_model)

In [ ]:
torch.save(resnext50_32x4d.state_dict(), f"{model_dir}/model_{best_acc:.4f}.pth")
np.save(f"{model_dir}/train_losses.npy" , np.array(train_losses))
np.save(f"{model_dir}/val_losses.npy" , np.array(val_losses))

train_epoch_step = len(train_losses)//len(val_losses)

train_plot_steps = np.arange(len(train_losses))+1
val_plot_steps = (np.arange(len(val_losses))+1)*train_epoch_step
plt.figure(figsize=(10,5))
plt.title("Losses")
plt.plot(train_plot_steps, train_losses, label='train_loss', linewidth=3)
plt.plot(val_plot_steps, val_losses, label='val_loss', linewidth=3)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig(f"{model_dir}/loss.png")
plt.show()

In [ ]:
resnext50_32x4d.eval()
num_images = 3

with torch.no_grad():
    val_inputs, val_labels = next(iter(dataloaders['aligned_val']))
    val_inputs = val_inputs.to(device)
    val_labels = val_labels.to(device)
    
    outputs = resnext50_32x4d(val_inputs)
    _, preds = torch.max(outputs, 1)
    
    for j in range(num_images):
        plt.figure(figsize=(8,8))
        plt.title(f"Actual:{class_names[val_labels[j]]}, Predicted:{class_names[preds[j]]}")
        plt.imshow((np.transpose(val_inputs[j].cpu().numpy(), (1, 2, 0))*np.array([0.229, 0.224, 0.225]))+np.array([0.485, 0.456, 0.406]))
        plt.savefig(f"{model_dir}/sample_predicted_{j}.png")